In [1]:
from gaspy.tasks.db_managers import catalog
from pymatgen import MPRester
from gaspy.utils import read_rc
from gaspy.tasks.core import schedule_tasks, get_task_output
from ast import literal_eval
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import read,write
from ase import Atoms, Atom
import catkit
from catkit.gen.adsorption import AdsorptionSites
from catkit.gen.surface import SlabGenerator
from ase.build import bulk
from ase import Atom
import numpy as np
matproj_api_key = 'B7pkHxCQtALPzGRy'

successfully loaded your custom FW_config.yaml!


# Catkit

In [2]:
def convert_atoms_to_gratoms(atoms, surface_atom_index):
    """
    Converts our slab documents into ase.Gratoms object
    Args:
    atoms:      ase atoms object
    Return:
    Gratoms:    ase graphic atoms object, which Catkit uses

    """
    gratoms = catkit.Gratoms(atoms)
    gratoms.set_surface_atoms(surface_atom_index)
    # our pbc default is [True, True, True], and that mess up adsorption sites
    # because of how CatKit is set up
    gratoms.pbc = np.array([True, True, False])
    return gratoms

In [9]:
import pickle
# Save processed data
file =  open('/global/project/projectdirs/m2755/sback/materials-project/03_analyze_coordination_number/all_docs_extended.pkl', 'rb')
docs = pickle.load(file)
file.close()

len(docs)

updated_docs = []

for i in docs:
    if len(i['structure']) < 50 and i['mpid'] != "mp-1120759":
         updated_docs.append(i)

docs = updated_docs
len(docs)

4860

In [311]:
# pdhg= []
# for i in updated_docs:
#     if i['mpid'] == 'mp-936':
#         pdhg.append(i)
        
# len(pdhg)

14

In [5]:
# for NiN
import pickle
# Save processed data
file =  open('/global/project/projectdirs/m2755/sback/materials-project/03_analyze_coordination_number/nin.pkl', 'rb')
docs = pickle.load(file)
file.close()

len(docs)

updated_docs = []

for i in docs:
    if len(i['structure']) < 50 and i['mpid'] != "mp-1120759":
         updated_docs.append(i)

docs = updated_docs
len(docs)

30

# Finding adsorption site. Test with pymatgen method (find_adsorption_sites)

In [312]:
# write('a.traj',AseAtomsAdaptor.get_atoms(docs[35]['structure']))

In [313]:
# import catkit
# n=2975
# struct = docs[n]['structure']
# atoms=AseAtomsAdaptor.get_atoms(docs[n]['structure'])
# sites_dict = AdsorbateSiteFinder(struct).find_adsorption_sites(put_inside=True)
# ads_atoms = []

# # Criteria 1:
# # Only consider top sites
# for site in sites_dict['ontop']:
#     adsorbate = Atoms('U')
#     adsorbate.translate(site)
#     adslab_atoms = atoms.copy() + adsorbate
#     adslab_atoms[-1].tag = 1 

#     # Let's find the index of binding site
#     U_index = [i.index for i in adslab_atoms if i.symbol == 'U'][0]
#     dist = adslab_atoms.get_all_distances(mic=True)[U_index]
    
#     for j,k in enumerate(dist):
#         if 1.99 < k < 2.01:
#             bindingsite_index = j
#     print(bindingsite_index)
    
#     gratom = convert_atoms_to_gratoms(atoms, bindingsite_index)
#     sites = AdsorptionSites(gratom)
#     coordinates = sites.get_coordinates()
#     vectors = sites.get_adsorption_vectors()
#     connectivity = sites.get_connectivity()
    
#     for coor, vec, conn in zip(coordinates, vectors, connectivity):
#     # Criteria 1:
#     # only consider top site binding because it should be isolate sites.
#         if conn == 1:
#             atoms_copy = atoms.copy()
#             atoms_copy += Atom('U', coor+vec*2)
#             ads_atoms.append(atoms_copy)
# len(ads_atoms)

In [314]:
# for i,j in enumerate(ads_atoms):
#     write(str(i)+'.traj', j)

In [7]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.analysis.local_env import VoronoiNN

inert = ['Au','S','Tl','Se','Hg','Ag','Br','C','Cl','F','I','O','N']

def add_adsorbate_isolated_site_pmg(index_docs):
    index = index_docs[0]
    doc = index_docs[1]

    
    ads_docs = []
    atoms=AseAtomsAdaptor.get_atoms(doc['structure'])
    struct = doc['structure']
    
    # Use pymatgen to find adsorption sites
    sites_dict = AdsorbateSiteFinder(struct).find_adsorption_sites(put_inside=True)
    ads_atoms = []
    
    try:
        # Criteria 1:
        # Only consider top sites 
        for site in sites_dict['ontop']:
            adsorbate = Atoms('U')
            adsorbate.translate(site)
            adslab_atoms = atoms.copy() + adsorbate
            adslab_atoms[-1].tag = 1 

            # Let's find the index of binding site
            U_index = [i.index for i in adslab_atoms if i.symbol == 'U'][0]
            dist = adslab_atoms.get_all_distances(mic=True)[U_index]
            for j,k in enumerate(dist):
                if 1.99 < k < 2.01:
                    bindingsite_index = j

            # Use catkit to place adsorbate far from the adsorption site
            gratom = convert_atoms_to_gratoms(atoms.copy(), bindingsite_index)
            sites = AdsorptionSites(gratom)
            coordinates = sites.get_coordinates()
            vectors = sites.get_adsorption_vectors()
            connectivity = sites.get_connectivity()

            for coor, vec, conn in zip(coordinates, vectors, connectivity):
                if conn == 1:
                    atoms_copy = atoms.copy()
                    atoms_copy += Atom('U', coor+vec*2)
                    U_index = [i.index for i in atoms_copy if i.symbol == 'U'][0]


            # Criteria 2:
            # collect adsorption geometry where the binding site is not inert atoms
            if atoms_copy[bindingsite_index].symbol not in inert:

                # Criteria 3:
                # nearest neighbor of binding site should have five nearest neighbors 
                vnn = VoronoiNN(allow_pathological=True)
                cn = vnn.get_nn_info(AseAtomsAdaptor.get_structure(atoms_copy), bindingsite_index)
                cn_index = [i['site_index'] for i in cn]
                cn_index.remove(U_index)
                sym = [str(AseAtomsAdaptor.get_structure(atoms_copy)[i].specie) for i in cn_index] # symbols of coordination atoms
                if len(sym) >= 5:
                    
                    # Criteria 4: at least four neighbors of inert elements:
                    # How many inert atoms near the active sites? if >= 4, accept
                    score1 = 0
                    for i in sym:
                        if i in inert:
                            score1 += 1
                    if score1 >= 4:
                        
                        # Criteria 5: at best two neighbors of same elements of binding sites:
                        if sym.count(atoms_copy[bindingsite_index].symbol) <=0: # 0? 1? 2?
                            ads_atoms.append(atoms_copy)            

        for i in ads_atoms:
                ads_doc = {}
                ads_doc['miller'] = doc['miller']
                ads_doc['structure'] = AseAtomsAdaptor.get_structure(i)
                ads_doc['top'] = doc['top']
                ads_doc['shift'] = doc['shift']
                ads_doc['mpid'] = doc['mpid']
                ads_docs.append(ads_doc)

    except ValueError:
        print(index)
    
    return ads_docs

# adsorption docs with Uranium atom (for O adsorbed geometries)

In [8]:
from joblib import Memory
import multiprocess as mp
from tqdm.notebook import tqdm

memory = Memory('./cached_docs', verbose=0)
docs_cached = memory.cache(add_adsorbate_isolated_site_pmg)

index_docs = []

for i, j in enumerate(docs):
    index_docs.append([i,j])
    
print(len(index_docs))

with mp.Pool(32) as pool:
       ads_docs = list(tqdm(pool.imap(docs_cached, index_docs), total=len(docs)))


# for i,j in enumerate(index_docs):
#     ads_docs = docs_cached(j)
#     print(i, len(ads_docs))


ads_docs_all = []
for i in ads_docs:
    for j in i:
        ads_docs_all.append(j)
        
print(len(ads_docs_all))

unique_mpids = []
for i in ads_docs_all:
    unique_mpids.append(i['mpid'])
unique_mpids = list(set(unique_mpids))
len(unique_mpids)

30



0


0

In [323]:
with open('/global/project/projectdirs/m2755/sback/materials-project/05_summarized_structures/ads_docs_all_0nearby.pkl', 'wb') as f:
    pickle.dump(ads_docs_all, f)

In [291]:
# o_atoms = []
# for i, doc in enumerate(ads_docs_all):
#     atoms=AseAtomsAdaptor.get_atoms(doc['structure']) # this structure has one Uranium atom
#     o_atoms.append(atoms)

# with open('/global/project/projectdirs/m2755/sback/materials-project/05_summarized_structures/o_atoms.pkl', 'wb') as f:
#     pickle.dump(o_atoms, f)

In [318]:
for i, doc in enumerate(ads_docs_all):
    atoms=AseAtomsAdaptor.get_atoms(doc['structure'])
    U_index = [i.index for i in atoms if i.symbol == 'U'][0]
    atoms[U_index].symbol = 'O'
    atoms[U_index].tag = 1
    write('/global/project/projectdirs/m2755/sback/materials-project/05_summarized_structures/o/'+str(i).zfill(3)+'.traj', atoms)
    
    

# adsorption docs with 2 Uranium and 1 Xenon (OOH* adsorbate)

In [319]:
import math
import numpy.linalg as LA

def make_ooh_using_cone(doc):
    
    atoms=AseAtomsAdaptor.get_atoms(doc['structure']) # this structure has one Uranium atom
    U_coord = [i.position for i in atoms if i.symbol=='U'][0]
    U_index = [i.index for i in atoms if i.symbol == 'U'][0]

    # Let's find the index of binding site
    dist = atoms.get_all_distances(mic=True)[U_index]
    for j,k in enumerate(dist):
        if 1.99 < k < 2.01: # because the distance between Uranium and the binding site is 2 angstrom.
            bindingsite_index = j # adsorption site

    # Use catkit to place adsorbate far from the adsorption site
    gratom = convert_atoms_to_gratoms(atoms.copy(), bindingsite_index)
    sites = AdsorptionSites(gratom)
    coordinates = sites.get_coordinates()
    vectors = sites.get_adsorption_vectors()
    connectivity = sites.get_connectivity()
    
    for coor, vec, conn in zip(coordinates, vectors, connectivity):
        if conn == 1:
            ads_vec = vec
            ads_coord = coor
            
    # New vector for additional O adsorption:
    ex_atoms = []
    
    for angle in np.arange(0, 360, 10): # generate 36 images
        i=math.sin(math.radians(30)) # 
        x=math.sqrt(1-i**2)*math.cos(math.radians(angle))
        y=math.sqrt(1-i**2)*math.sin(math.radians(angle))
        z=i
        new_vac1 = np.array([x,y,z]) # for O
        
        jj=math.sin(math.radians(60))
        xx=math.sqrt(1-jj**2)*math.cos(math.radians(angle))
        yy=math.sqrt(1-jj**2)*math.sin(math.radians(angle))
        zz=jj
        new_vac2 = np.array([xx,yy,zz]) # for H

        atoms_copy = atoms.copy()
#        atoms_copy[U_index].symbol = 'O'
        atoms_copy += Atom('Fr',  U_coord+1.414*(new_vac1)) # O-O distance to be 1.414
        atoms_copy += Atom('Xe', U_coord+1.9*new_vac2) # to make O-H distance to be ~1
        ex_atoms.append(atoms_copy)
    
    dist = []
    for tmp in ex_atoms:
        Fr_index = [i.index for i in tmp if i.symbol == 'Fr'][0] 
        Xe_index = [i.index for i in tmp if i.symbol == 'Xe'][0]
        dist.append(sum(tmp.get_all_distances(mic=True)[U_index] + tmp.get_all_distances(mic=True)[Xe_index]))
    max_index = max(enumerate(dist), key=itemgetter(1))[0]

    new_atoms = ex_atoms[max_index]

#     # temporarilty save trajectories
#     for i,j in enumerate(ex_atoms):
#         for k,l in enumerate(j):
#             if l.symbol == 'U':
#                 j[k].symbol = 'O'
#                 j[k].tag = 1
#             elif l.symbol =='Xe':
#                 j[k].symbol = 'H'
#                 j[k].tag = 1
#         write('test/'+str(i).zfill(2)+'.traj', j)


    # Change atoms to OOH for the best geometry
#     for i,j in enumerate(new_atoms):
#         if j.symbol == 'U':
#             new_atoms[i].symbol = 'O'
#             new_atoms[i].tag = 1
#         elif j.symbol =='Xe':
#             new_atoms[i].symbol = 'H'
#             new_atoms[i].tag = 1


    ooh_doc = {}
    ooh_doc['miller'] = doc['miller']
    ooh_doc['structure'] = AseAtomsAdaptor.get_structure(new_atoms)
    ooh_doc['top'] = doc['top']
    ooh_doc['shift'] = doc['shift']
    ooh_doc['mpid'] = doc['mpid']

    return ooh_doc        
  

In [321]:
for i, doc in enumerate(ads_docs_all):
    doc=make_ooh_using_cone(doc)
    atoms=AseAtomsAdaptor.get_atoms(doc['structure'])
    U_index = [i.index for i in atoms if i.symbol == 'U'][0]
    Fr_index = [i.index for i in atoms if i.symbol == 'Fr'][0] 
    Xe_index = [i.index for i in atoms if i.symbol == 'Xe'][0]
    atoms[U_index].symbol = 'O'
    atoms[U_index].tag = 1
    atoms[Fr_index].symbol = 'O'
    atoms[Fr_index].tag = '1'
    atoms[Xe_index].symbol = 'H'
    atoms[Xe_index].tag = '1'

    write('/global/project/projectdirs/m2755/sback/materials-project/05_summarized_structures/ooh/'+str(i).zfill(3)+'.traj', atoms)
    
    
    

In [206]:
from operator import itemgetter

def make_ooh_catalog_from_Uranium(doc):
#    index = index_docs[0]
#    doc = index_docs[1]
    atoms=AseAtomsAdaptor.get_atoms(doc['structure'])
    

    # Generate 4 rotational OOH* adsorbed geometries. Instead of using OOH*, use Uranium for O and Xenon for H. 
    tmp = []
    U_coord = [i.position for i in atoms if i.symbol=='U'][0]
    atoms_copy0 = atoms.copy() 
    atoms_copy0 += Atom('U',  U_coord+[1,0,1])
    atoms_copy0 += Atom('Xe', U_coord+[0.5,0,1.86])
    atoms_copy1 = atoms.copy() 
    atoms_copy1 += Atom('U',  U_coord+[0,1,1])
    atoms_copy1 += Atom('Xe', U_coord+[0,0.5,1.86])
    atoms_copy2 = atoms.copy() 
    atoms_copy2 += Atom('U',  U_coord+[-1,0,1])
    atoms_copy2 += Atom('Xe', U_coord+[-0.5,0,1.86])
    atoms_copy3 = atoms.copy() 
    atoms_copy3 += Atom('U',  U_coord+[0,-1,1])
    atoms_copy3 += Atom('Xe', U_coord+[0,-0.5,1.86])
    
    atoms_copy4 = atoms.copy() 
    atoms_copy4 += Atom('U',  U_coord+[1.414,0,0])
    atoms_copy4 += Atom('Xe', U_coord+[1.75,0,0.95])
    atoms_copy5 = atoms.copy() 
    atoms_copy5 += Atom('U',  U_coord+[0,1.414,0])
    atoms_copy5 += Atom('Xe', U_coord+[0,1.75,0.95])
    atoms_copy6 = atoms.copy() 
    atoms_copy6 += Atom('U',  U_coord+[-1.414,0,0])
    atoms_copy6 += Atom('Xe', U_coord+[-1.75,0,0.95])
    atoms_copy7 = atoms.copy() 
    atoms_copy7 += Atom('U',  U_coord+[0,-1.414,0])
    atoms_copy7 += Atom('Xe', U_coord+[0,-1.75,0.95])
    
    atoms_copy8 = atoms.copy() 
    atoms_copy8 += Atom('U',  U_coord+[0,-1,-1])
    atoms_copy8 += Atom('Xe', U_coord+[0,-1.95,-0.55])
    atoms_copy9 = atoms.copy() 
    atoms_copy9 += Atom('U',  U_coord+[1,0,-1])
    atoms_copy9 += Atom('Xe', U_coord+[1.95,0,-0.55])
    atoms_copy10 = atoms.copy() 
    atoms_copy10 += Atom('U',  U_coord+[0,1,-1])
    atoms_copy10 += Atom('Xe', U_coord+[0,1.95,-0.55])
    atoms_copy11 = atoms.copy() 
    atoms_copy11 += Atom('U',  U_coord+[-1,0,-1])
    atoms_copy11 += Atom('Xe', U_coord+[-1.95,0,-0.55])
    
    tmps = [atoms_copy0, atoms_copy1, atoms_copy2, atoms_copy3, 
           atoms_copy4, atoms_copy5, atoms_copy6, atoms_copy7, 
           atoms_copy8, atoms_copy9, atoms_copy10, atoms_copy11]
    
    dist = []
    for tmp in tmps:
        U_index = [i.index for i in tmp if i.symbol == 'U'][1] # Second Uranium
        Xe_index = [i.index for i in tmp if i.symbol == 'Xe'][0]
        dist.append(sum(tmp.get_all_distances(mic=True)[U_index] + tmp.get_all_distances(mic=True)[Xe_index]))
    max_index = max(enumerate(dist), key=itemgetter(1))[0]
    
    new_atoms = tmps[max_index]
    
    # Comment later
    for i,j in enumerate(new_atoms):
        if j.symbol == 'U':
            new_atoms[i].symbol = 'O'
            new_atoms[i].tag = 1
        elif j.symbol =='Xe':
            new_atoms[i].symbol = 'H'
            new_atoms[i].tag = 1
    
    
    ooh_doc = {}
    ooh_doc['miller'] = doc['miller']
    ooh_doc['structure'] = AseAtomsAdaptor.get_structure(new_atoms)
    ooh_doc['top'] = doc['top']
    ooh_doc['shift'] = doc['shift']
    ooh_doc['mpid'] = doc['mpid']
    
    return ooh_doc        
        

 

In [91]:
dft_mpids = ['mp-865144', 'mp-1095440', 'mp-979269', 'mp-569683', 'mp-22216', 'mp-982246', 'mp-21276', 'mp-30721',
            'mp-20250', 'mp-936', 'mp-1101251', 'mp-174']

for i in dft_mpids:
    if i in list(set(unique_mpids)):
        print(str(i)+" is in the list")
    else:
        print(str(i)+" is not in the list")

mp-865144 is not in the list
mp-1095440 is not in the list
mp-979269 is not in the list
mp-569683 is in the list
mp-22216 is not in the list
mp-982246 is not in the list
mp-21276 is not in the list
mp-30721 is not in the list
mp-20250 is not in the list
mp-936 is in the list
mp-1101251 is not in the list
mp-174 is not in the list


In [92]:
import random

random_docs = random.sample(ads_docs_all, 50)

for i,j in enumerate(random_docs):
    write('test/'+str(i)+'.traj',AseAtomsAdaptor.get_atoms(make_ooh_catalog_from_Uranium(j)['structure']))

# Below is different approach (using CatKit)

# Using CatKit, find isolated single sites

In [111]:
from pymatgen.io.ase import AseAtomsAdaptor
import catkit
from ase.io import read
from ase.data import covalent_radii as CR
import sys
from catkit.gen.adsorption import AdsorptionSites, Builder
from datetime import datetime
import numpy as np
import os
from ase.io import read, write
from pymatgen.io.ase import AseAtomsAdaptor
from ase import Atom

## Find surface sites (how to identify surface atoms?) 
It's important to find isolated sites now, to be used to generate adsobate-adsorbed geometries

In [112]:
# def get_coordination_numbers(atoms, covalent_percent=1.25):
#     """Returns an array of coordination numbers and an array of existing bonds determined by
#     distance and covalent radii.  By default a bond is defined as 120% of the combined radii
#     or less. This can be changed by setting 'covalent_percent' to a float representing a 
#     factor to multiple by (default = 1.2).
#     If 'exclude' is set to an array,  these atomic numbers with be unable to form bonds.
#     This only excludes them from being counted from other atoms,  the coordination
#     numbers for these atoms will still be calculated,  but will be unable to form
#     bonds to other excluded atomic numbers.
#     """

#     # Get all the distances
#     distances = np.divide(atoms.get_all_distances(mic=True), covalent_percent)
    
#     # Atomic Numbers
#     numbers = atoms.numbers
#     # Coordination Numbers for each atom
#     cn = []
#     cr = np.take(CR, numbers)
#     # Array of indices of bonded atoms.  len(bonded[x]) == cn[x]
#     bonded = []
#     indices = list(range(len(atoms)))
#     for i in indices:
#         bondedi = []
#         for ii in indices:
#             # Skip if measuring the same atom
#             if i == ii:
#                 continue
#             if (cr[i] + cr[ii]) >= distances[i,ii]:
#                 bondedi.append(ii)
#         # Add this atoms bonds to the bonded list
#         bonded.append(bondedi)
#     for i in bonded:
#         cn.append(len(i))
    
#     # Dictionary of coordination numbers per atom index
#     cn_dict = {}
#     for i,j in enumerate(cn):
#         cn_dict[i] = j
    
#     # Dictionary of coordination number per elements
#     cn_elem_dict = {}
    
#     for i,j in enumerate(atoms):
#         elem = j.symbol
#         if elem not in cn_elem_dict.keys():
#             cn_elem_dict[elem] = []
#         cn_elem_dict[elem].append(cn_dict[i])
    
#     # Find maximum coordination numbers per elements
#     max_cn_dict = {}
#     min_cn_dict = {}
#     for i in cn_elem_dict.keys():
#         max_cn_dict[i] = max(cn_elem_dict[i])
#         min_cn_dict[i] = min(cn_elem_dict[i])

#     # Assign surface atoms
#     surface_atom_index = []
#     for i,j in enumerate(atoms):
#         if atoms.get_scaled_positions()[i][2] > 0.5:
#             # This condition is very important. Learn from DFT calculations
#             if max_cn_dict[j.symbol] > cn_dict[i] >= max_cn_dict[j.symbol] - 3 :#or cn_dict[i] < min_cn_dict[j.symbol]+3:
# #            if cn_dict[i] < min_cn_dict[j.symbol] + 5:
#                 surface_atom_index.append(i)
    
#     return cn, bonded, cn_dict, cn_elem_dict, surface_atom_index

In [113]:
# def get_cn_voronoi(struct):
#     cnn = CrystalNN(distance_cutoffs=None, x_diff_weight=0.0, porous_adjustment=False)
#     data = []
#     for index, atom in enumerate(struct):
#         data.append([index, atom.specie, cnn.get_cn_dict(struct, index)])
    
#     cn_dict = {}
#     for i in data:
#         cn_dict[i[0]] = sum(i[2].values())

#     cn_elem_dict = {}
#     for i,j in enumerate(struct):
#         elem = str(j.specie)
#         if elem not in cn_elem_dict.keys():
#             cn_elem_dict[elem] = []
#         cn_elem_dict[elem].append(cn_dict[i])
    
#     # Find maximum coordination numbers per elements
#     max_cn_dict = {}
#     min_cn_dict = {}
#     for i in cn_elem_dict.keys():
#         max_cn_dict[i] = max(cn_elem_dict[i])
#         min_cn_dict[i] = min(cn_elem_dict[i])

#     # Assign surface atoms
#     surface_atom_index = []
#     for i,j in enumerate(atoms):
#         if atoms.get_scaled_positions()[i][2] > 0.5:
#             # This condition is very important. Learn from DFT calculations
#             if max_cn_dict[j.symbol] > cn_dict[i] >= max_cn_dict[j.symbol] - 3 or cn_dict[i] < min_cn_dict[j.symbol] + 3:
#                 surface_atom_index.append(i)
    
#     return cn_dict, cn_elem_dict, surface_atom_index

# Use this to find surface atoms (from pymatgen and modified by Seoin) (to be used in CatKit)

In [4]:
from pymatgen.analysis.local_env import VoronoiNN
import math
def find_cn_voronoiNN(struct): 
# Get a dictionary of coordination numbers
# for eaziph distinct site in the structure


    cn_dict = {}
    v = VoronoiNN()
    unique_indices = [i for i in range(len(struct))]

    for i in unique_indices:
        el = struct[i].species_string
        if el not in cn_dict.keys():
            cn_dict[el] = []
        # Since this will get the cn as a result of the weighted polyhedra, the
        # slightest difference in cn will indicate a different environment for a
        # species, eg. bond distance of each neighbor or neighbor species. The
        # decimal place to get some cn to be equal.
        cn = v.get_cn(struct, i, use_weights=True)
        cn = float('%.5f' % (round(cn, 3)))
        if cn not in cn_dict[el]:
            cn_dict[el].append(cn)

    v = VoronoiNN()

    surf_sites_dict, properties = {"top": [], "bottom": []}, []
    for i, site in enumerate(struct):
        # Determine if site is closer to the top or bottom of the slab
        center_of_mass = np.average(struct.frac_coords, axis=0)          
        top = site.frac_coords[2] > center_of_mass[2]

        try:
            # A site is a surface site, if its environment does
            # not fit the environment of other sites
            cn = float('%.5f' % (round(v.get_cn(struct, i, use_weights=True), 3)))
            if cn < (max(cn_dict[site.species_string]) + min(cn_dict[site.species_string]))/2: # Seoin modified
                properties.append(True)
                key = "top" if top else "bottom"
                surf_sites_dict[key].append([site, i])
            else:
                properties.append(False)
        except RuntimeError:
            # or if pathological error is returned, indicating a surface site
            properties.append(True)
            key = "top" if top else "bottom"
            surf_sites_dict[key].append([site, i])
            
    return surf_sites_dict

In [163]:
# # Test for my cases

# import random
# import pandas as pd
# random = random.sample(range(len(docs)), 10)
# surface_atom_indexes=[]

# for i in random:
#     atoms=AseAtomsAdaptor.get_atoms(docs[i]['structure'])
#     write('test/'+str(i)+'.traj', atoms)
#     struct = AseAtomsAdaptor.get_structure(atoms)
#     surf_sites_dict = find_cn_voronoiNN(struct)
#     surface_atom_index = []
#     for j in surf_sites_dict['top']:
#         surface_atom_index.append(j[-1])
#     name.append(i)
#     surface_atom_indexes.append(surface_atom_index)

    
# sample = pd.DataFrame()
# sample['index'] = random
# sample['surface_atom_index'] = surface_atom_indexes

# sample

,index,surface_atom_index
0,3882,"[9, 11, 13, 14]"
1,2298,"[2, 5, 9, 13, 14]"
2,889,"[1, 10]"
3,2466,"[1, 2, 4, 5, 6, 7, 8, 9, 10, 13, 14, 17, 18, 20]"
4,5026,"[36, 37, 38, 39, 46, 47]"
5,5305,"[31, 32, 33, 34, 35, 39, 40, 47, 49, 52]"
6,1005,"[2, 6]"
7,2713,"[9, 10, 14, 15]"
8,6319,"[32, 34]"
9,3320,"[0, 5, 6, 8, 10]"


# Catkit

In [116]:
def convert_atoms_to_gratoms(atoms, surface_atom_index):
    """
    Converts our slab documents into ase.Gratoms object
    Args:
    atoms:      ase atoms object
    Return:
    Gratoms:    ase graphic atoms object, which Catkit uses

    """
    gratoms = catkit.Gratoms(atoms)
    gratoms.set_surface_atoms(surface_atom_index)
    # our pbc default is [True, True, True], and that mess up adsorption sites
    # because of how CatKit is set up
    gratoms.pbc = np.array([True, True, False])
    return gratoms

# Add adsorbate at the isolated sites that satisfy certain conditions (1. top site binding, 2. binding site is not inert atoms, 3. four nearest neighbor inert atoms)

In [136]:
inert = ['Au','S','Tl','Se','Hg','Ag']

def add_adsorbate_isolated_site(index_docs):
    index = index_docs[0]
    doc = index_docs[1]
    
    ads_docs = []
    atoms=AseAtomsAdaptor.get_atoms(doc['structure'])
#    write('test.traj', atoms)
    struct = doc['structure']
    surf_sites_dict = find_cn_voronoiNN(struct)
    surface_atom_index = [i[-1] for i in surf_sites_dict['top']]
    
    try:
        gratom = convert_atoms_to_gratoms(atoms, surface_atom_index)
        sites = AdsorptionSites(gratom)
        coordinates = sites.get_coordinates()
        vectors = sites.get_adsorption_vectors()
        connectivity = sites.get_connectivity()

        ads_atoms = []
        sites = []
        for coor, vec, conn in zip(coordinates, vectors, connectivity):
            # Criteria 1:
            # only consider top site binding because it should be isolate sites.
            if conn == 1:
                atoms_copy = atoms.copy()
                atoms_copy += Atom('U', coor+vec*2)
                U_index = [i.index for i in atoms_copy if i.symbol == 'U'][0]
                dist = atoms_copy.get_all_distances()[U_index]
                dist_dict = {}
                for j,k in enumerate(dist):
                    dist_dict[j] = k
                sorted_dist_dict = {k: v for k, v in sorted(dist_dict.items(), key=lambda item: item[1])}
                bindingsite_index = [i for i in sorted_dist_dict.keys()][1]

                # Criteria 2:
                # collect adsorption geometry where the binding site is not inert atoms
                if atoms_copy[bindingsite_index].symbol not in inert:

                    # Criteria 3:
                    # nearest neighbor of binding site should have at least four neighbors of other element:
                    dist = atoms_copy.get_all_distances()[bindingsite_index]
                    dist_dict = {}
                    for j,k in enumerate(dist):
                        dist_dict[j] = k
                    sorted_dist_dict = {k: v for k, v in sorted(dist_dict.items(), key=lambda item: item[1])}
                    
                    # Find indexes of five NN atoms (not "U", not itself)
                    five_NN_atoms_index = [i for i in sorted_dist_dict.keys()][2:7]
                    sym = []
                    for i in five_NN_atoms_index:
                        sym.append(atoms_copy[i].symbol)
                    
                    # How many inert atoms near the active sites? if >= 4, accept
                    measure = 0
                    for i in sym:
                        if i in inert:
                            measure += 1
                    if measure >= 4:
                        ads_atoms.append(atoms_copy)
#                     # If all are inert atoms, accept (four atoms)
#                     if all([i in inert for i in sym ]):
#                         ads_atoms.append(atoms_copy)

        for i in ads_atoms:
            ads_doc = {}
            ads_doc['miller'] = doc['miller']
            ads_doc['structure'] = AseAtomsAdaptor.get_structure(i)
            ads_doc['top'] = doc['top']
            ads_doc['shift'] = doc['shift']
            ads_doc['mpid'] = doc['mpid']
            ads_docs.append(ads_doc)
        
    except ValueError:
        print(index)

        
    return ads_docs
                
    

In [137]:
from joblib import Memory
import multiprocess as mp
from tqdm.notebook import tqdm

memory = Memory('./cached_docs', verbose=0)
docs_cached = memory.cache(add_adsorbate_isolated_site)

index_docs = []

for i, j in enumerate(docs):
    index_docs.append([i,j])

with mp.Pool(32) as pool:
       ads_docs = list(tqdm(pool.imap(docs_cached, index_docs), total=len(docs)))


# for i,j in enumerate(docs):
#     ads_docs = docs_cached(j)
#     print(i, len(ads_docs))

15
20
46
48
51
57
113
119
142
144
122
125
149
191
197
213
344
347
321
318
324
462
464
472
513
516
510
680
677
746
750
791
834
830
961
962
967
968
975
970
1083
1088
1094
1102
1114
1116
1152
1139
1147
1226
1236
1429
1432
1435
1698
1703
1718
1723
1724
1782
1877
1883
1880
1908
1729
1992
2011
2008
2047
2050
2044
2146
2144
2148
2150
2154
2152
2122
2123
2156
2230
2329
2339
2337
2377
2379
2388
2481
2510
2506
2503
2531
2540
2541
2546
2685
2691
2694
2764
2767
2770
2795
2816
2826
2840
2745
3002
3067
3070
3113
3116
3119
3541
3557
3618
3621
3710
3712
3713
3715
3719
3721
3968
3971
3975
4015
3978
4019
4093
4095
4139
4145
4133
4157
4161
4163
4152
4220
4223
4226
4324
4434
4489
4494
4623
4599
4603
4641
4646
5125
5128
5132
5130
5134
5141
5215
5212
5224
5229
5314
5317
5311
5517
5606
5609
5616
5648
5691
5667
5804
5817
5816



In [139]:
ads_docs_all = []
for i in ads_docs:
    for j in i:
        ads_docs_all.append(j)
        
print(len(ads_docs_all))

unique_mpids = []
for i in ads_docs_all:
    unique_mpids.append(i['mpid'])
    
len(set(unique_mpids))

3965


144

In [141]:
import random

random_docs = random.sample(ads_docs_all, 50)

for i,j in enumerate(random_docs):
    write(str(i)+'.traj',AseAtomsAdaptor.get_atoms(j['structure']))

In [103]:
struct = docs[802]['structure']
atoms=AseAtomsAdaptor.get_atoms(struct)
#    write('test.traj', atoms)

surf_sites_dict = find_cn_voronoiNN(struct)
surface_atom_index = [i[-1] for i in surf_sites_dict['top']]
gratom = convert_atoms_to_gratoms(atoms, surface_atom_index)
sites = AdsorptionSites(gratom)
coordinates = sites.get_coordinates()
vectors = sites.get_adsorption_vectors()
connectivity = sites.get_connectivity()

In [90]:
surface_atom_index

[8, 10, 18, 21]

In [63]:
struct = docs[15]['structure']
cn_dict = {}
v = VoronoiNN()
unique_indices = [i for i in range(len(struct))]

for i in unique_indices:
    el = struct[i].species_string
    if el not in cn_dict.keys():
        cn_dict[el] = []
    # Since this will get the cn as a result of the weighted polyhedra, the
    # slightest difference in cn will indicate a different environment for a
    # species, eg. bond distance of each neighbor or neighbor species. The
    # decimal place to get some cn to be equal.
    cn = v.get_cn(struct, i, use_weights=True)
    cn = float('%.5f' % (round(cn, 3)))
    if cn not in cn_dict[el]:
        cn_dict[el].append(cn)

v = VoronoiNN()

surf_sites_dict, properties = {"top": [], "bottom": []}, []
for i, site in enumerate(struct):
    # Determine if site is closer to the top or bottom of the slab
    center_of_mass = np.average(struct.frac_coords, axis=0)          
    top = site.frac_coords[2] > center_of_mass[2]

    try:
        # A site is a surface site, if its environment does
        # not fit the environment of other sites
        cn = float('%.5f' % (round(v.get_cn(struct, i, use_weights=True), 3)))
        prin
        if cn < 0.5*(max(cn_dict[site.species_string]) + min(cn_dict[site.species_string])): # Seoin modified
            properties.append(True)
            key = "top" if top else "bottom"
            surf_sites_dict[key].append([site, i])
        else:
            properties.append(False)
    except RuntimeError:
        # or if pathological error is returned, indicating a surface site
        properties.append(True)
        key = "top" if top else "bottom"
        surf_sites_dict[key].append([site, i])

0 4.815
1 7.592
2 6.371
3 7.592
4 4.211
5 7.592
6 6.689
7 7.592
8 6.371
9 7.592
10 6.447
11 6.614
12 6.833
13 7.235
14 7.592
15 5.784
16 4.211
17 7.592
18 7.592
19 4.557


In [61]:
surf_sites_dict

{'top': [],
 'bottom': [[PeriodicSite: Pd (1.2352, 5.1442, 13.2503) [0.2324, 0.6584, 0.4093],
   0],
  [PeriodicSite: Pd (4.6361, 1.7839, 12.4300) [0.8723, 0.2283, 0.3918], 4],
  [PeriodicSite: Pd (3.3361, 6.8978, 14.0549) [0.6277, 0.8828, 0.4304], 15],
  [PeriodicSite: Se (2.6574, 3.2954, 12.2469) [0.5000, 0.4218, 0.3821], 16],
  [PeriodicSite: Se (0.0000, 7.2022, 12.5631) [-0.0000, 0.9218, 0.3821], 19]]}

In [65]:
cn_dict['Pd']

[4.815, 7.592, 6.371, 4.211, 6.689, 6.447, 6.614, 6.833, 7.235, 5.784]

In [59]:
min(cn_dict['Pd']), max(cn_dict['Pd']), 0.5*(min(cn_dict['Pd'])+max(cn_dict['Pd']))

(4.211, 7.592, 5.9015)

# Below are useless

In [151]:
# Test for DFT cases

import pandas as pd
from pymatgen.analysis.local_env import CrystalNN
pd.set_option('display.max_colwidth', -1)

dft_path = '/global/homes/s/sback/m2755/sback/materials-project/04_find_isolated_sites/unique_prototypes/'
name=[]
surface_atom_indexes=[]


for i in os.listdir(dft_path):
    atoms=read(dft_path+str(i)+'/input.POSCAR')
    struct = AseAtomsAdaptor.get_structure(atoms)
    surf_sites_dict = find_cn_voronoiNN(struct)
    surface_atom_index = []
    for j in surf_sites_dict['top']:
        surface_atom_index.append(j[-1])
    name.append(i)
    surface_atom_indexes.append(surface_atom_index)

    
sample = pd.DataFrame()
sample['name'] = name
sample['surface_atom_index'] = surface_atom_indexes



sample

,name,surface_atom_index
0,PbTl3,"[5, 6, 9, 10]"
1,CdSe2,"[9, 11, 13, 14, 16, 17]"
2,Tl2Pt,"[10, 11, 12, 13, 15, 16]"
3,InHg3,"[8, 9, 10, 11]"
4,InAu3,"[4, 6, 7, 10]"
5,Tl2Pd,"[10, 11, 12, 13, 15, 16]"
6,Te3Tl5_1,"[8, 9, 10, 11, 12, 13]"
7,PdAu3,"[4, 7, 8, 11]"
8,Hg4Pt,"[4, 6, 8, 10, 17, 18]"
9,PdS,"[5, 9, 10, 13, 14]"


In [164]:
# based on bond length and covalent radii


# pd.set_option('display.max_colwidth', -1)

# dft_path = '/global/homes/s/sback/m2755/sback/materials-project/04_find_isolated_sites/unique_prototypes/'
# name=[]
# surface_atom_indexes=[]
# cn_elem = []

# for i in os.listdir(dft_path):
#     atoms=read(dft_path+str(i)+'/input.POSCAR')
#     name.append(i)
#     cn, bonded, cn_dict, cn_elem_dict, surface_atom_index = get_coordination_numbers(atoms)
#     surface_atom_indexes.append(surface_atom_index)
#     cn_elem.append(str(cn_elem_dict))
    
# sample = pd.DataFrame()
# sample['name'] = name

# sample['surface_atom_index'] = surface_atom_indexes
# sample['cn_elem_dict'] = cn_elem


# sample

# Test for DFT calculated results

In [18]:
dft_path = "/global/homes/s/sback/m2755/sback/materials-project/my_dft_bulk/"
ase_atoms = []
pymatgen_structures = []
name = []


for i in os.listdir(dft_path):
    if i!='summary':
        name.append(i)
        atoms = read(dft_path+str(i)+'/CONTCAR')
        ase_atoms.append(atoms)
        pymatgen_structures.append(AseAtomsAdaptor.get_structure(atoms))
    
cn_covalent = []
for i in ase_atoms:
    cn_covalent.append(get_coordination_numbers(i))
    

In [19]:
for i,j in zip(name, ase_atoms):
    write(dft_path+"/summary/"+str(i)+".cif", j)

In [20]:
active_atom = ['In', 'Pb', 'Pt', 'Te', 'Cd', 'Pd', 'Ag']



In [210]:
from pymatgen.analysis.local_env import CrystalNN

CrystalNN = CrystalNN(distance_cutoffs=None, x_diff_weight=0.0, porous_adjustment=False)
cn_pymatgen = []

for pymatgen_structure in pymatgen_structures:
    active_atoms = []
    cn = []
    active_atom_symbols = []
    for i,j in enumerate(pymatgen_structure):
        if str(j.specie) in active_atom:
            active_atoms.append(i)
    for i in active_atoms:
        cn.append(CrystalNN.get_cn_dict(pymatgen_structure, i))
        active_atom_symbols.append(str(pymatgen_structure[i].specie))
    cn_pymatgen.append([active_atoms, cn, active_atom_symbols])
        


In [213]:
df_dft = pd.DataFrame()
df_dft['name'] = name
df_dft['active_atom_index'] = [i[0] for i in cn_pymatgen]
df_dft['active_atom_symbol'] = [i[2] for i in cn_pymatgen]
df_dft['cn'] = [i[1] for i in cn_pymatgen]



In [214]:
df_dft

,name,active_atom_index,active_atom_symbol,cn
0,PbTl3,"[6, 7]","[Pb, Pb]","[{'Tl': 12}, {'Tl': 12}]"
1,CdSe2,"[0, 1, 2, 3]","[Cd, Cd, Cd, Cd]","[{'Se': 6}, {'Se': 6}, {'Se': 6}, {'Se': 6}]"
2,Tl2Pt,"[8, 9, 10, 11]","[Pt, Pt, Pt, Pt]","[{'Tl': 4, 'Pt': 2}, {'Tl': 4, 'Pt': 2}, {'Tl'..."
3,InHg3,"[0, 1]","[In, In]","[{'Hg': 12}, {'Hg': 12}]"
4,InAu3,"[0, 1]","[In, In]","[{'Au': 12}, {'Au': 12}]"
5,Tl2Pd,"[8, 9, 10, 11]","[Pd, Pd, Pd, Pd]","[{'Tl': 4, 'Pd': 2}, {'Tl': 4, 'Pd': 2}, {'Tl'..."
6,PdAu3,"[0, 1]","[Pd, Pd]","[{'Au': 12}, {'Au': 12}]"
7,Hg4Pt,"[8, 9]","[Pt, Pt]","[{'Hg': 8}, {'Hg': 8}]"
8,PdS,"[0, 1, 2, 3, 4, 5, 6, 7]","[Pd, Pd, Pd, Pd, Pd, Pd, Pd, Pd]","[{'S': 4}, {'S': 4}, {'S': 4}, {'S': 4}, {'S':..."
9,PtS,"[0, 1, 2, 3, 4, 5, 6, 7]","[Pt, Pt, Pt, Pt, Pt, Pt, Pt, Pt]","[{'S': 4}, {'S': 4}, {'S': 4}, {'S': 4}, {'S':..."
